In [34]:
import pandas as pd

### Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [3]:
def get_class(x):
    res = ''
    if(x<=2): res = 'низкий рейтинг'
    elif(x>=3 and x<=4): res = 'средний рейтинг'
    elif(x>=4.5 and x<=5): res = 'высокий рейтинг'
    return res

ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings['class'] = ratings['rating'].apply(get_class) # Применяем фунцию для получения стоблца class
joined = ratings.merge(movies, on='movieId', how='left') # Объединяем таблицы
joined.head()

,userId,movieId,rating,timestamp,class,title,genres
0,1,1,4.0,964982703,средний рейтинг,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,средний рейтинг,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,средний рейтинг,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,высокий рейтинг,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,высокий рейтинг,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


### Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [69]:
def get_region(words, geo_data):
    region = 'undefined'
    for key,row in geo_data.items(): # Для каждого региона
        for city in row: # И каждого горjда
            if words.find(city)>0: # Ищем совпадение с названием города
                region=key # И возвращаем регион
    return region
    
    
geo_data = pd.DataFrame({
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
})
geo_data.to_dict() # Переводим в словарь для удобства

keywords = pd.read_csv('keywords.csv')
keywords['region'] = keywords['keyword'].apply(get_region, geo_data=geo_data) # Применяем фунцию к каждой строке
keywords[keywords['region']=='Дальний Восток'].head()

,keyword,shows,region
1063,фарпост владивосток,176951,Дальний Восток
2175,дром хабаровск,98361,Дальний Восток
3206,дром владивосток,73084,Дальний Восток
3275,100 сп владивосток,76418,Дальний Восток
3687,авито хабаровск,63736,Дальний Восток


### Задание 3 (бонусное)
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

Важно! При копировании юпитер ноутбука изменяет кавычки ’ на ` , что может способствовать ошибки в коде. В данной ситуации необходимо их просто заменить.

In [116]:
def production_year(title, years):
    result='1900'
    for year in years:
        if title.find(f"({year})")>0:
            result = str(year)
    return result

ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')

years = list(range(1950,2011)) # Получаем список годов с 1950 по 2010 (указан 2011, потому что берется до него)
movies['year'] = movies['title'].apply(production_year, years=years)
joined = ratings.merge(movies, on='movieId', how='left') # Объединяем таблицы
joined.groupby('year').agg({'rating': 'mean'}).sort_values(['rating'], ascending=[False]) # Получаем средний рейтинг

,rating
year,
1957,4.039535
1954,4.009191
1962,3.969466
1952,3.953125
1964,3.940160
...,...
2000,3.398899
2005,3.359888
1992,3.354943
